# Lab 05: Multivariate Methods
**BSAD 8310: Business Forecasting — University of Nebraska at Omaha**

## Objectives
1. Test for Granger causality between consumer sentiment and retail sales
2. Fit a VAR(p) model and plot impulse response functions
3. Fit an ARIMAX model with an exogenous predictor
4. Compare forecast accuracy across ARIMA, VAR, and ARIMAX

## Dataset
- **RSXFS**: Advance Retail Sales — Retail and Food Services (monthly, SA, millions USD)
- **UMCSENT**: University of Michigan: Consumer Sentiment (monthly index)
- Source: FRED (Federal Reserve Bank of St. Louis)
- Fallback: statsmodels macrodata (quarterly GDP + income)

In [ ]:
# ── 1. Setup ──────────────────────────────────────────────────────────────────
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from pathlib import Path

np.random.seed(42)

# UNO color palette
UNO_BLUE  = '#005CA9'
UNO_RED   = '#E41C38'
UNO_GRAY  = '#525252'
UNO_GREEN = '#15803d'

plt.rcParams.update({
    'figure.dpi': 150,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.titlesize': 11,
    'axes.labelsize': 10,
    'legend.fontsize': 9,
})

FIGURES = Path('../Figures')
FIGURES.mkdir(exist_ok=True)
print('Setup complete.')

In [ ]:
# ── 2. Load Data ──────────────────────────────────────────────────────────────
try:
    import pandas_datareader.data as web
    from datetime import datetime
    start, end = '2000-01-01', '2023-12-31'
    rsxfs  = web.DataReader('RSXFS',   'fred', start, end)
    umcsent= web.DataReader('UMCSENT', 'fred', start, end)
    df = pd.concat([rsxfs, umcsent], axis=1).dropna()
    df.columns = ['retail', 'sentiment']
    df.index = pd.DatetimeIndex(df.index).to_period('M')
    print(f'FRED data loaded: {len(df)} months, {df.index[0]} to {df.index[-1]}')
    FREQ = 'M'
except Exception as e:
    print(f'FRED unavailable ({e}); using statsmodels macrodata fallback.')
    import statsmodels.api as sm
    macro = sm.datasets.macrodata.load_pandas().data
    macro.index = pd.period_range('1959Q1', periods=len(macro), freq='Q')
    df = macro[['realgdp', 'realcons']].copy()
    df.columns = ['retail', 'sentiment']   # rename for uniform code
    print(f'Fallback data loaded: {len(df)} quarters, {df.index[0]} to {df.index[-1]}')
    FREQ = 'Q'

print(df.describe().round(1))

In [ ]:
# ── 3. Exploratory Analysis ───────────────────────────────────────────────────
fig, axes = plt.subplots(3, 1, figsize=(10, 8))

# Raw series
axes[0].plot(df.index.to_timestamp(), df['retail'],    color=UNO_BLUE, lw=1.4, label='Retail Sales')
axes[0].set_title('Retail Sales (RSXFS)')
axes[0].set_ylabel('USD millions')

axes[1].plot(df.index.to_timestamp(), df['sentiment'], color=UNO_RED,  lw=1.4, label='Consumer Sentiment')
axes[1].set_title('Consumer Sentiment (UMCSENT)')
axes[1].set_ylabel('Index')

# Cross-correlation function
from statsmodels.tsa.stattools import ccf
r_diff = df['retail'].diff().dropna()
s_diff = df['sentiment'].diff().dropna()
n_ccf  = min(len(r_diff), len(s_diff))
r_diff = r_diff.iloc[-n_ccf:].values
s_diff = s_diff.iloc[-n_ccf:].values
lags   = np.arange(-12, 13)
ccf_vals = [np.corrcoef(s_diff[:n_ccf-abs(k)], r_diff[abs(k):] if k <= 0
             else r_diff[:n_ccf-k])[0,1] for k in lags]
ci = 1.96 / np.sqrt(n_ccf)

axes[2].bar(lags, ccf_vals, color=[UNO_BLUE if abs(v) > ci else UNO_GRAY
            for v in ccf_vals], width=0.6)
axes[2].axhline(ci,  color=UNO_GRAY, ls='--', lw=0.8)
axes[2].axhline(-ci, color=UNO_GRAY, ls='--', lw=0.8)
axes[2].axvline(0, color='black', lw=0.5)
axes[2].set_xlabel('Lag k (sentiment leads at negative k)')
axes[2].set_ylabel('CCF')
axes[2].set_title('Cross-Correlation: Sentiment (leads) vs. Retail Changes')

for ax in axes: ax.spines[['top','right']].set_visible(False)
plt.tight_layout()
plt.savefig(FIGURES / 'lecture05_eda.png', bbox_inches='tight')
plt.show()
print('Figure saved.')

In [ ]:
# ── 4. Unit Root Tests ────────────────────────────────────────────────────────
from statsmodels.tsa.stattools import adfuller, kpss

def unit_root_summary(series, name):
    adf_stat, adf_p, *_ = adfuller(series.dropna(), autolag='AIC')
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        kpss_stat, kpss_p, *_ = kpss(series.dropna(), regression='c', nlags='auto')
    decision = ('I(1): unit root likely'
                if adf_p > 0.05 and kpss_p < 0.05 else
                'I(0): stationary'
                if adf_p < 0.05 and kpss_p > 0.05 else
                'Ambiguous')
    print(f'{name}: ADF p={adf_p:.3f}, KPSS p={kpss_p:.3f}  →  {decision}')
    return decision

print('=== Unit root tests (levels) ===')
d_retail    = unit_root_summary(df['retail'],    'Retail')
d_sentiment = unit_root_summary(df['sentiment'], 'Sentiment')

print('\n=== Unit root tests (first differences) ===')
unit_root_summary(df['retail'].diff(),    'ΔRetail')
unit_root_summary(df['sentiment'].diff(), 'ΔSentiment')

In [ ]:
# ── 5. Granger Causality Test ─────────────────────────────────────────────────
from statsmodels.tsa.stattools import grangercausalitytests

# Use first differences (assume I(1) series)
df_diff = df.diff().dropna()

print('H₀: Sentiment does NOT Granger-cause Retail')
print('─' * 50)
gc_results = grangercausalitytests(
    df_diff[['retail', 'sentiment']], maxlag=4, verbose=False
)
for lag, res in gc_results.items():
    f_stat = res[0]['ssr_ftest'][0]
    p_val  = res[0]['ssr_ftest'][1]
    sig    = '***' if p_val < 0.01 else '**' if p_val < 0.05 else '*' if p_val < 0.10 else ''
    print(f'  Lag {lag}: F = {f_stat:.2f}, p = {p_val:.4f} {sig}')

print('\nH₀: Retail does NOT Granger-cause Sentiment')
print('─' * 50)
gc_results2 = grangercausalitytests(
    df_diff[['sentiment', 'retail']], maxlag=4, verbose=False
)
for lag, res in gc_results2.items():
    f_stat = res[0]['ssr_ftest'][0]
    p_val  = res[0]['ssr_ftest'][1]
    sig    = '***' if p_val < 0.01 else '**' if p_val < 0.05 else '*' if p_val < 0.10 else ''
    print(f'  Lag {lag}: F = {f_stat:.2f}, p = {p_val:.4f} {sig}')

In [ ]:
# ── 6. VAR Model and Impulse Response Functions ───────────────────────────────
from statsmodels.tsa.api import VAR

# Train/test split: hold out last 12 periods
H = 12
train_diff = df_diff.iloc[:-H]
test_diff  = df_diff.iloc[-H:]

# Fit VAR, select order by BIC
var_model  = VAR(train_diff)
order_sel  = var_model.select_order(maxlags=8)
p_bic      = order_sel.bic
print(f'BIC-selected VAR order: p = {p_bic}')

var_fit = var_model.fit(p_bic)
print(var_fit.summary())

# Impulse Response Functions
irf = var_fit.irf(periods=12)

fig, axes = plt.subplots(1, 2, figsize=(11, 4))
for ax, response_idx, title in zip(
    axes,
    [0, 0],
    ['Response of Retail to Retail Shock',
     'Response of Retail to Sentiment Shock']
):
    pass  # overwrite below

ax0, ax1 = axes
horizons = np.arange(13)

irf_vals_retail_retail   = irf.irfs[:, 0, 0]
irf_vals_retail_sentiment= irf.irfs[:, 0, 1]

ax0.bar(horizons, irf_vals_retail_retail,    color=UNO_BLUE, alpha=0.8)
ax0.axhline(0, color='black', lw=0.6)
ax0.set_title('Retail ← Retail shock')
ax0.set_xlabel('Horizon')

ax1.bar(horizons, irf_vals_retail_sentiment, color=UNO_RED,  alpha=0.8)
ax1.axhline(0, color='black', lw=0.6)
ax1.set_title('Retail ← Sentiment shock')
ax1.set_xlabel('Horizon')

for ax in axes: ax.spines[['top','right']].set_visible(False)
plt.suptitle('Impulse Response Functions (VAR)', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig(FIGURES / 'lecture05_irf.png', bbox_inches='tight')
plt.show()
print('IRF figure saved.')

In [ ]:
# ── 7. ARIMAX Model ───────────────────────────────────────────────────────────
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Levels for original series; use differenced sentiment as exogenous
y_train = df['retail'].iloc[1:-H]         # skip first (lost to diff)
x_train = df_diff['sentiment'].iloc[:-H]
y_test  = df['retail'].iloc[-H:]
x_test  = df_diff['sentiment'].iloc[-H:]

# Align indices
min_len = min(len(y_train), len(x_train))
y_train = y_train.iloc[-min_len:]
x_train = x_train.iloc[-min_len:]

# Fit ARIMAX(1,1,1) with contemporaneous sentiment change
arimax_fit = SARIMAX(
    y_train,
    exog=x_train.values.reshape(-1,1),
    order=(1, 1, 1),
    seasonal_order=(0, 1, 1, 12),
    enforce_stationarity=False,
    enforce_invertibility=False
).fit(disp=False)

print(arimax_fit.summary().tables[1])
print(f'\nAIC: {arimax_fit.aic:.1f}   BIC: {arimax_fit.bic:.1f}')

In [ ]:
# ── 8. Forecast Comparison ────────────────────────────────────────────────────
from statsmodels.tsa.statespace.sarimax import SARIMAX as _SARIMAX

# Benchmark: SARIMA (no exogenous)
arima_fit = _SARIMAX(
    y_train,
    order=(1, 1, 1),
    seasonal_order=(0, 1, 1, 12),
    enforce_stationarity=False,
    enforce_invertibility=False
).fit(disp=False)

# ARIMA forecast
arima_fc = arima_fit.forecast(H)

# ARIMAX forecast (need future exogenous values — use test set)
arimax_fc = arimax_fit.forecast(H, exog=x_test.values.reshape(-1,1))

# VAR forecast (on differenced data, then cumsum back to levels)
var_fc_diff = var_fit.forecast(train_diff.values[-p_bic:], steps=H)
last_level  = df['retail'].iloc[-(H+1)]
var_fc_levels = last_level + np.cumsum(var_fc_diff[:, 0])

# Naive forecast
naive_fc = np.full(H, float(y_train.iloc[-1]))

y_true = y_test.values

def rmse(y, yhat): return np.sqrt(np.mean((y - yhat)**2))
def mae(y, yhat):  return np.mean(np.abs(y - yhat))

results = pd.DataFrame({
    'Model':  ['Naive', 'SARIMA', 'ARIMAX', 'VAR'],
    'RMSE':   [rmse(y_true, naive_fc),
               rmse(y_true, arima_fc.values),
               rmse(y_true, arimax_fc.values),
               rmse(y_true, var_fc_levels)],
    'MAE':    [mae(y_true, naive_fc),
               mae(y_true, arima_fc.values),
               mae(y_true, arimax_fc.values),
               mae(y_true, var_fc_levels)],
}).set_index('Model').round(1)

print('Forecast accuracy (hold-out last 12 periods):')
print(results)

In [ ]:
# ── 9. Forecast Visualization ─────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(11, 5))

# Historical (last 36 periods)
hist = df['retail'].iloc[-(H+36):-H]
ax.plot(hist.index.to_timestamp(), hist.values,
        color=UNO_GRAY, lw=1.2, label='History')

test_idx = y_test.index.to_timestamp()

ax.plot(test_idx, y_true,             color='black',    lw=1.5, ls='-',  label='Actual')
ax.plot(test_idx, naive_fc,           color=UNO_GRAY,   lw=1.2, ls='--', label='Naive')
ax.plot(test_idx, arima_fc.values,    color=UNO_BLUE,   lw=1.4, ls='-',  label='SARIMA')
ax.plot(test_idx, arimax_fc.values,   color=UNO_GREEN,  lw=1.4, ls='-',  label='ARIMAX')
ax.plot(test_idx, var_fc_levels,      color=UNO_RED,    lw=1.4, ls='-',  label='VAR')

# ARIMAX 95% PI
fc_summary = arimax_fit.get_forecast(H, exog=x_test.values.reshape(-1,1))
ci = fc_summary.conf_int(alpha=0.05)
ax.fill_between(test_idx,
                ci.iloc[:, 0], ci.iloc[:, 1],
                color=UNO_GREEN, alpha=0.15, label='ARIMAX 95% PI')

ax.axvline(test_idx[0], color='black', lw=0.8, ls=':')
ax.set_title('Retail Sales Forecast Comparison', fontsize=13, fontweight='bold')
ax.set_ylabel('USD millions')
ax.legend(ncol=3, fontsize=8)
ax.spines[['top','right']].set_visible(False)

plt.tight_layout()
plt.savefig(FIGURES / 'lecture05_forecasts.png', bbox_inches='tight')
plt.show()
print('Forecast figure saved.')